In [ ]:
%%writefile frontend/.gitignore
# Byte-compiled / optimized / DLL files
__pycache__/
*.py[cod]
*$py.class
.env.example
# C extensions
*.so

# Distribution / packaging
.Python
build/
develop-eggs/
dist/
downloads/
eggs/
.eggs/
lib/
lib64/
parts/
sdist/
var/
wheels/
share/python-wheels/
*.egg-info/
.installed.cfg
*.egg
MANIFEST

# PyInstaller
#  Usually these files are written by a python script from a template
#  before PyInstaller builds the exe, so as to inject date/other infos into it.
*.manifest
*.spec

# Installer logs
pip-log.txt
pip-delete-this-directory.txt

# Unit test / coverage reports
htmlcov/
.tox/
.nox/
.coverage
.coverage.*
.cache
nosetests.xml
coverage.xml
*.cover
*.py,cover
.hypothesis/
.pytest_cache/
cover/

# Translations
*.mo
*.pot

# Django stuff:
*.log
local_settings.py
db.sqlite3
db.sqlite3-journal

# Flask stuff:
instance/
.webassets-cache

# Scrapy stuff:
.scrapy

# Sphinx documentation
docs/_build/

# PyBuilder
.pybuilder/
target/

# Jupyter Notebook
.ipynb_checkpoints

# IPython
profile_default/
ipython_config.py

# pyenv
#   For a library or package, you might want to ignore these files since the code is
#   intended to run in multiple environments; otherwise, check them in:
# .python-version

# pipenv
#   According to pypa/pipenv#598, it is recommended to include Pipfile.lock in version control.
#   However, in case of collaboration, if having platform-specific dependencies or dependencies
#   having no cross-platform support, pipenv may install dependencies that don't work, or not
#   install all needed dependencies.
#Pipfile.lock

# poetry
#   Similar to Pipfile.lock, it is generally recommended to include poetry.lock in version control.
#   This is especially recommended for binary packages to ensure reproducibility, and is more
#   commonly ignored for libraries.
#   https://python-poetry.org/docs/basic-usage/#commit-your-poetrylock-file-to-version-control
#poetry.lock

# pdm
#   Similar to Pipfile.lock, it is generally recommended to include pdm.lock in version control.
#pdm.lock
#   pdm stores project-wide configurations in .pdm.toml, but it is recommended to not include it
#   in version control.
#   https://pdm.fming.dev/#use-with-ide
.pdm.toml

# PEP 582; used by e.g. github.com/David-OConnor/pyflow and github.com/pdm-project/pdm
__pypackages__/

# Celery stuff
celerybeat-schedule
celerybeat.pid

# SageMath parsed files
*.sage.py

# Environments
.env
.venv
env/
venv/
ENV/
env.bak/
venv.bak/

# Spyder project settings
.spyderproject
.spyproject

# Rope project settings
.ropeproject

# mkdocs documentation
/site

# mypy
.mypy_cache/
.dmypy.json
dmypy.json

# Pyre type checker
.pyre/

# pytype static type analyzer
.pytype/

# Cython debug symbols
cython_debug/

# PyCharm
#  JetBrains specific template is maintained in a separate JetBrains.gitignore that can
#  be found at https://github.com/github/gitignore/blob/main/Global/JetBrains.gitignore
#  and can be added to the global gitignore or merged into this file.  For a more nuclear
#  option (not recommended) you can uncomment the following to ignore the entire idea folder.
#.idea/


In [ ]:
%%writefile Dockerfile.railway
# Railway Frontend Dockerfile
# =========================
# This Dockerfile implements an optimized build process with proper caching:
# - Uses explicit cache paths to avoid conflicts with Railway's build system
# - Implements multi-stage build for better layer caching
# - Separates dependency installation from build for better caching

# BUILD STAGE
FROM node:18-alpine AS build

# Set working directory
WORKDIR /app

# Copy package files first (for better layer caching)
COPY package*.json ./

# Install dependencies
# We avoid mounting any caches under node_modules to prevent EBUSY errors
RUN npm ci --prefer-offline --no-audit --loglevel=error

# Copy source code
COPY . .

# Build with environment variables
# Railway provides these at build time, but we need to make sure they're available
ARG VITE_API_URL
ENV VITE_API_URL=$VITE_API_URL

# Debug: Show what environment variables are available during build
RUN echo "🔍 BUILD DEBUG: VITE_API_URL = $VITE_API_URL"
RUN echo "🔍 BUILD DEBUG: All ENV vars:" && env | grep -E "(VITE_|NODE_|RAILWAY_)" || true

# Build the application with verbose output
RUN echo "🔍 BUILD DEBUG: Starting build process..." && \
    npm run build 2>&1 | tee build.log

# Debug: Check if the build contains the correct API URL
RUN echo "🔍 BUILD DEBUG: Checking built files for API URL..." && \
    find dist -name "*.js" -exec grep -l "fastapi-production-1d13.up.railway.app" {} \; || \
    echo "🔍 BUILD DEBUG: API URL not found in built files"

# Debug: Show actual content of built files (first 50 lines)
RUN echo "🔍 BUILD DEBUG: Sample content from built JS files:" && \
    find dist -name "*.js" | head -1 | xargs head -50 || \
    echo "🔍 BUILD DEBUG: No JS files found"

# Debug: Search for any occurrence of 'VITE_API_URL' in built files
RUN echo "🔍 BUILD DEBUG: Searching for VITE_API_URL in built files:" && \
    find dist -name "*.js" -exec grep -H "VITE_API_URL" {} \; || \
    echo "🔍 BUILD DEBUG: VITE_API_URL not found in built files"

# SERVE STAGE
FROM node:18-alpine

WORKDIR /app

# Install serve globally
RUN npm install -g serve

# Copy only the built assets from build stage
COPY --from=build /app/dist ./dist

# Expose port
EXPOSE $PORT

# Start the application
CMD ["sh", "-c", "serve -s dist -l $PORT --no-clipboard --no-port-switching"] 

In [ ]:
%%writefile frontend/eslint.config.js
import js from '@eslint/js'
import globals from 'globals'
import reactHooks from 'eslint-plugin-react-hooks'
import reactRefresh from 'eslint-plugin-react-refresh'
import tseslint from 'typescript-eslint'

export default tseslint.config(
  { ignores: ['dist'] },
  {
    extends: [js.configs.recommended, ...tseslint.configs.recommended],
    files: ['**/*.{ts,tsx}'],
    languageOptions: {
      ecmaVersion: 2020,
      globals: globals.browser,
    },
    plugins: {
      'react-hooks': reactHooks,
      'react-refresh': reactRefresh,
    },
    rules: {
      ...reactHooks.configs.recommended.rules,
      'react-refresh/only-export-components': [
        'warn',
        { allowConstantExport: true },
      ],
    },
  },
)


In [ ]:
%%writefile frontend/index.html
<!doctype html>
<html lang="en">
  <head>
    <meta charset="UTF-8" />
    <link rel="icon" type="image/svg+xml" href="/vite.svg" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0" />
    <title>Vite + React + TS</title>
  </head>
  <body>
    <div id="root"></div>
    <script type="module" src="/src/main.tsx"></script>
  </body>
</html>


In [ ]:
%%writefile frontend/nixpacks.toml
[phases.setup]
nixPkgs = ["nodejs-18_x", "npm-9_x"]

[phases.install]
cmds = [
  "npm ci --prefer-offline --no-audit --loglevel=error"
]

[phases.build]
cmds = [
  "npm run build"
]

[start]
cmd = "npx serve -s dist -l $PORT --no-clipboard --no-port-switching"

[variables]
NODE_ENV = "production"
NPM_CONFIG_PRODUCTION = "false" 

In [ ]:
%%writefile frontend/package.json
{
  "name": "vite-react-typescript-starter",
  "private": true,
  "version": "0.0.0",
  "type": "module",
  "scripts": {
    "dev": "vite",
    "build": "tsc -b && vite build",
    "build:railway": "npm ci --prefer-offline --no-audit --loglevel=error && npm run build",
    "lint": "eslint .",
    "preview": "vite preview"
  },
  "dependencies": {
    "react": "^19.0.0",
    "react-dom": "^19.0.0"
  },
  "devDependencies": {
    "@eslint/js": "^9.21.0",
    "@types/node": "^22.0.0",
    "@types/react": "^19.0.10",
    "@types/react-dom": "^19.0.4",
    "@vitejs/plugin-react": "^4.3.4",
    "eslint": "^9.21.0",
    "eslint-plugin-react-hooks": "^5.1.0",
    "eslint-plugin-react-refresh": "^0.4.19",
    "globals": "^15.15.0",
    "serve": "^14.2.3",
    "typescript": "~5.7.2",
    "typescript-eslint": "^8.24.1",
    "vite": "^6.2.0"
  }
}


In [ ]:
%%writefile frontend/railway.alternative.json
{
  "$schema": "https://railway.app/railway.schema.json",
  "build": {
    "builder": "DOCKERFILE",
    "dockerfilePath": "Dockerfile.railway"
  },
  "deploy": {
    "startCommand": "npx serve -s dist -l $PORT --no-clipboard --no-port-switching",
    "restartPolicyType": "ON_FAILURE",
    "restartPolicyMaxRetries": 10
  }
} 

In [ ]:
%%writefile frontend/railway.json
{
  "$schema": "https://railway.app/railway.schema.json",
  "build": {
    "builder": "DOCKERFILE",
    "dockerfilePath": "Dockerfile.railway"
  },
  "deploy": {
    "restartPolicyType": "ON_FAILURE",
    "restartPolicyMaxRetries": 10
  },
  "environments": {
    "production": {
      "variables": {
        "VITE_API_URL": "https://fastapi-production-1d13.up.railway.app"
      }
    }
  }
}


In [ ]:
%%writefile frontend/tsconfig.app.json
{
  "compilerOptions": {
    "tsBuildInfoFile": "./node_modules/.tmp/tsconfig.app.tsbuildinfo",
    "target": "ES2020",
    "useDefineForClassFields": true,
    "lib": ["ES2020", "DOM", "DOM.Iterable"],
    "module": "ESNext",
    "skipLibCheck": true,

    /* Bundler mode */
    "moduleResolution": "bundler",
    "allowImportingTsExtensions": true,
    "isolatedModules": true,
    "moduleDetection": "force",
    "noEmit": true,
    "jsx": "react-jsx",

    /* Linting */
    "strict": true,
    "noUnusedLocals": true,
    "noUnusedParameters": true,
    "noFallthroughCasesInSwitch": true,
    "noUncheckedSideEffectImports": true
  },
  "include": ["src"]
}


In [ ]:
%%writefile frontend/tsconfig.json
{
  "files": [],
  "references": [
    { "path": "./tsconfig.app.json" },
    { "path": "./tsconfig.node.json" }
  ]
}


In [ ]:
%%writefile frontend/vite.config.ts
import { defineConfig, loadEnv } from 'vite'
import react from '@vitejs/plugin-react'

// https://vite.dev/config/
export default defineConfig(({ mode }) => {
  const env = loadEnv(mode, process.cwd(), '')
  
  // API URL configuration
  // Development: local backend
  // Production: Railway backend URL
  const API_URL = env.VITE_API_URL || 
    (mode === 'production' 
      ? 'https://fastapi-production-1d13.up.railway.app' 
      : 'http://127.0.0.1:8000')

  console.log('🔍 Vite Config Debug:')
  console.log('Mode:', mode)
  console.log('API_URL:', API_URL)
  console.log('VITE_API_URL from env:', env.VITE_API_URL)
  console.log('All VITE_ env vars:', Object.keys(env).filter(key => key.startsWith('VITE_')))
  console.log('NODE_ENV:', process.env.NODE_ENV)
  console.log('PWD:', process.env.PWD)
  console.log('Railway vars:', {
    RAILWAY_ENVIRONMENT: process.env.RAILWAY_ENVIRONMENT,
    RAILWAY_PROJECT_ID: process.env.RAILWAY_PROJECT_ID,
    RAILWAY_SERVICE_NAME: process.env.RAILWAY_SERVICE_NAME
  })

  return {
    plugins: [react()],
    server: {
      host: '0.0.0.0',
      port: 5173,
      proxy: {
        '/api': {
          target: API_URL,
          changeOrigin: true,
          secure: false,
          rewrite: (path) => path, // Keep the /api prefix
          configure: (proxy, _options) => {
            proxy.on('error', (err, _req, _res) => {
              console.log('🔍 Proxy Error:', err)
            })
            proxy.on('proxyReq', (proxyReq, req, _res) => {
              console.log('🔍 Proxy Request:', req.method, req.url, '-> ', proxyReq.path)
            })
            proxy.on('proxyRes', (proxyRes, req, _res) => {
              console.log('🔍 Proxy Response:', proxyRes.statusCode, req.url)
            })
          }
        }
      }
    },
    build: {
      outDir: 'dist',
      assetsDir: 'assets',
      sourcemap: false,
      rollupOptions: {
        output: {
          manualChunks: {
            vendor: ['react', 'react-dom']
          }
        }
      }
    },
    define: {
      __API_URL__: JSON.stringify(API_URL)
    }
  }
})


In [ ]:
%%writefile frontend/src/App.css
#root {
  max-width: 1280px;
  margin: 0 auto;
  padding: 2rem;
  text-align: center;
}

.logo {
  height: 6em;
  padding: 1.5em;
  will-change: filter;
  transition: filter 300ms;
}
.logo:hover {
  filter: drop-shadow(0 0 2em #646cffaa);
}
.logo.react:hover {
  filter: drop-shadow(0 0 2em #61dafbaa);
}

@keyframes logo-spin {
  from {
    transform: rotate(0deg);
  }
  to {
    transform: rotate(360deg);
  }
}

@media (prefers-reduced-motion: no-preference) {
  a:nth-of-type(2) .logo {
    animation: logo-spin infinite 20s linear;
  }
}

.card {
  padding: 2em;
}

.read-the-docs {
  color: #888;
}


In [ ]:
%%writefile frontend/src/App.tsx
import { useState, useEffect } from 'react'
import reactLogo from './assets/react.svg'
import viteLogo from '/vite.svg'
import './App.css'

interface ApiResponse {
  message: string
}

interface AppInfo {
  app_name: string
  version: string
  environment: string
  railway_environment?: string
  python_version: string
}

interface PredictionRequest {
  data: Record<string, any>
}

interface PredictionResponse {
  prediction: any
  confidence: number
  model_version: string
}

function App() {
  const [count, setCount] = useState(0)
  const [apiMessage, setApiMessage] = useState<string>('')
  const [appInfo, setAppInfo] = useState<AppInfo | null>(null)
  const [isLoading, setIsLoading] = useState(false)
  const [error, setError] = useState<string>('')
  const [predictionResult, setPredictionResult] = useState<PredictionResponse | null>(null)

  // Define backend URL with multiple fallback layers
  const getBackendUrl = () => {
    // First try: Environment variable from build time
    if (import.meta.env.VITE_API_URL) {
      return import.meta.env.VITE_API_URL
    }
    
    // Second try: Runtime detection (if we're in production)
    if (import.meta.env.PROD) {
      return 'https://fastapi-production-1d13.up.railway.app'
    }
    
    // Third try: Development fallback
    return 'http://127.0.0.1:8000'
  }
  
  const BACKEND_URL = getBackendUrl()
  
  const getApiUrl = (endpoint: string) => {
    // In development, use proxy path (Vite handles /api routing)
    if (import.meta.env.DEV) {
      console.log('🔍 DEV MODE: Using proxy path:', endpoint)
      return endpoint
    }
    
    // In production, ALWAYS use absolute URL
    let backendUrl = BACKEND_URL
    
    // Ensure we have a valid URL
    if (!backendUrl || backendUrl === '' || backendUrl === 'undefined') {
      console.warn('🔍 FALLBACK: Backend URL is invalid, using hardcoded fallback')
      backendUrl = 'https://fastapi-production-1d13.up.railway.app'
    }
    
    // Clean the backend URL (remove trailing slash)
    const cleanBackendUrl = backendUrl.replace(/\/$/, '')
    
    // Ensure endpoint starts with /
    const cleanEndpoint = endpoint.startsWith('/') ? endpoint : `/${endpoint}`
    
    // Construct the full URL
    const fullUrl = `${cleanBackendUrl}${cleanEndpoint}`
    
    console.log('🔍 PROD MODE: Constructing URL:', {
      originalBackendUrl: BACKEND_URL,
      backendUrl,
      cleanBackendUrl,
      endpoint,
      cleanEndpoint,
      fullUrl,
      isAbsolute: fullUrl.startsWith('http')
    })
    
    // Final validation - ensure we have an absolute URL
    if (!fullUrl.startsWith('http')) {
      console.error('🔍 ERROR: Generated URL is not absolute!', fullUrl)
      const fallbackUrl = `https://fastapi-production-1d13.up.railway.app${cleanEndpoint}`
      console.warn('🔍 FALLBACK: Using hardcoded URL:', fallbackUrl)
      return fallbackUrl
    }
    
    return fullUrl
  }

  console.log('🔍 App Debug Info:')
  console.log('Mode:', import.meta.env.MODE)
  console.log('DEV:', import.meta.env.DEV)
  console.log('PROD:', import.meta.env.PROD)
  console.log('BACKEND_URL:', BACKEND_URL)
  
  // Add comprehensive debugging
  console.log('🔍 COMPREHENSIVE DEBUG INFO:')
  console.log('import.meta.env.VITE_API_URL:', import.meta.env.VITE_API_URL)
  console.log('typeof import.meta.env.VITE_API_URL:', typeof import.meta.env.VITE_API_URL)
  console.log('BACKEND_URL length:', BACKEND_URL.length)
  console.log('BACKEND_URL === "":', BACKEND_URL === '')
  console.log('Sample API URL generation:')
  console.log('  /api/hello ->', getApiUrl('/api/hello'))
  console.log('  /api/predict ->', getApiUrl('/api/predict'))
  console.log('Current window.location:', window.location.href)
  console.log('All import.meta.env keys:', Object.keys(import.meta.env))
  console.log('All import.meta.env values:', import.meta.env)

  // Runtime URL validation
  const validateApiUrl = (url: string) => {
    try {
      const urlObj = new URL(url)
      const isValid = urlObj.protocol === 'https:' || urlObj.protocol === 'http:'
      console.log('🔍 URL VALIDATION:', {
        url,
        isValid,
        protocol: urlObj.protocol,
        hostname: urlObj.hostname,
        pathname: urlObj.pathname
      })
      return isValid
    } catch (e) {
      console.error('🔍 URL VALIDATION ERROR:', { url, error: e })
      return false
    }
  }

  // Validate our generated URLs
  useEffect(() => {
    const testUrls = ['/api/hello', '/api/info', '/api/predict']
    testUrls.forEach(endpoint => {
      const generatedUrl = getApiUrl(endpoint)
      validateApiUrl(generatedUrl)
    })
  }, [])

  // Enhanced fetch with better error handling
  const fetchWithDebug = async (url: string, options: RequestInit = {}) => {
    console.log('🔍 FETCH DEBUG: Starting request to:', url)
    console.log('🔍 FETCH DEBUG: Request options:', options)
    
    try {
      const response = await fetch(url, options)
      console.log('🔍 FETCH DEBUG: Response received:', {
        status: response.status,
        statusText: response.statusText,
        headers: Object.fromEntries(response.headers.entries()),
        url: response.url
      })
      
      return response
    } catch (error) {
      console.error('🔍 FETCH DEBUG: Network error:', error)
      throw error
    }
  }

  // Fetch API message on component mount
  useEffect(() => {
    // Run URL generation test first
    testUrlGeneration()
    
    fetchApiMessage()
    fetchAppInfo()
  }, [])

  const fetchApiMessage = async () => {
    try {
      setIsLoading(true)
      setError('')
      const url = getApiUrl('/api/hello')
      console.log('🔍 DEBUG: Fetching from URL:', url)
      
      // Validate URL before making request
      if (!validateApiUrl(url)) {
        throw new Error(`Invalid URL generated: ${url}`)
      }
      
      const response = await fetchWithDebug(url)
      if (!response.ok) {
        throw new Error(`HTTP error! status: ${response.status}`)
      }
      const data: ApiResponse = await response.json()
      setApiMessage(data.message)
    } catch (err) {
      setError(`Failed to fetch API message: ${err instanceof Error ? err.message : 'Unknown error'}`)
      console.error('Error fetching API message:', err)
    } finally {
      setIsLoading(false)
    }
  }

  const fetchAppInfo = async () => {
    try {
      const url = getApiUrl('/api/info')
      console.log('🔍 DEBUG: Fetching app info from URL:', url)
      
      // Validate URL before making request
      if (!validateApiUrl(url)) {
        console.error('Invalid URL generated for app info:', url)
        return
      }
      
      const response = await fetchWithDebug(url)
      if (!response.ok) {
        throw new Error(`HTTP error! status: ${response.status}`)
      }
      const data: AppInfo = await response.json()
      setAppInfo(data)
    } catch (err) {
      console.error('Error fetching app info:', err)
    }
  }

  const testPrediction = async () => {
    try {
      setIsLoading(true)
      setError('')
      const requestData: PredictionRequest = {
        data: {
          feature1: Math.random(),
          feature2: Math.random(),
          feature3: count
        }
      }
      
      const url = getApiUrl('/api/predict')
      console.log('🔍 DEBUG: Making prediction request to:', url)
      console.log('🔍 DEBUG: Request data:', requestData)
      
      const response = await fetchWithDebug(url, {
        method: 'POST',
        headers: {
          'Content-Type': 'application/json',
        },
        body: JSON.stringify(requestData)
      })
      
      console.log('🔍 DEBUG: Response status:', response.status)
      console.log('🔍 DEBUG: Response headers:', Object.fromEntries(response.headers.entries()))
      
      if (!response.ok) {
        // Get the response text to see what error we're getting
        const errorText = await response.text()
        console.log('🔍 DEBUG: Error response text:', errorText)
        throw new Error(`HTTP error! status: ${response.status}, response: ${errorText}`)
      }
      
      // Get the response as text first to see what we're actually getting
      const responseText = await response.text()
      console.log('🔍 DEBUG: Raw response text:', responseText)
      
      // Try to parse it as JSON
      let data: PredictionResponse
      try {
        data = JSON.parse(responseText)
        console.log('🔍 DEBUG: Successfully parsed JSON:', data)
      } catch (parseError) {
        console.error('🔍 DEBUG: JSON parse error:', parseError)
        console.error('🔍 DEBUG: Response text that failed to parse:', responseText)
        throw new Error(`Failed to parse JSON response: ${parseError instanceof Error ? parseError.message : 'Unknown parse error'}. Response was: ${responseText}`)
      }
      
      setPredictionResult(data)
    } catch (err) {
      setError(`Prediction failed: ${err instanceof Error ? err.message : 'Unknown error'}`)
      console.error('🔍 DEBUG: Full error object:', err)
    } finally {
      setIsLoading(false)
    }
  }

  // Add a diagnostic function to test the API connection
  const runDiagnostics = async () => {
    console.log('🔍 RUNNING API DIAGNOSTICS...')
    
    // Test 1: Check what URL we're actually calling
    const testUrl = getApiUrl('/api/health')
    console.log('🔍 DIAGNOSTIC: Health check URL:', testUrl)
    
    try {
      const response = await fetchWithDebug(testUrl)
      console.log('🔍 DIAGNOSTIC: Health check response status:', response.status)
      console.log('🔍 DIAGNOSTIC: Health check response headers:', Object.fromEntries(response.headers.entries()))
      
      const text = await response.text()
      console.log('🔍 DIAGNOSTIC: Health check response text:', text)
      
      // Try to parse as JSON
      try {
        const json = JSON.parse(text)
        console.log('🔍 DIAGNOSTIC: Health check JSON:', json)
      } catch (e) {
        console.log('🔍 DIAGNOSTIC: Health check response is not JSON')
      }
    } catch (error) {
      console.error('🔍 DIAGNOSTIC: Health check failed:', error)
    }
    
    // Test 2: Check CORS preflight
    console.log('🔍 DIAGNOSTIC: Testing CORS preflight...')
    try {
      const corsResponse = await fetchWithDebug(testUrl, {
        method: 'OPTIONS',
        headers: {
          'Access-Control-Request-Method': 'GET',
          'Access-Control-Request-Headers': 'Content-Type',
        }
      })
      console.log('🔍 DIAGNOSTIC: CORS preflight status:', corsResponse.status)
      console.log('🔍 DIAGNOSTIC: CORS preflight headers:', Object.fromEntries(corsResponse.headers.entries()))
    } catch (error) {
      console.error('🔍 DIAGNOSTIC: CORS preflight failed:', error)
    }
  }

  // Comprehensive URL testing function
  const testUrlGeneration = () => {
    console.log('🔍 TESTING URL GENERATION...')
    
    const testEndpoints = [
      '/api/hello',
      '/api/info',
      '/api/health',
      '/api/predict',
      'api/hello', // without leading slash
      '/api/docs'
    ]
    
    testEndpoints.forEach(endpoint => {
      const generatedUrl = getApiUrl(endpoint)
      const isValid = validateApiUrl(generatedUrl)
      
      console.log('🔍 URL TEST:', {
        endpoint,
        generatedUrl,
        isValid,
        isAbsolute: generatedUrl.startsWith('http'),
        containsBackend: generatedUrl.includes('fastapi-production-1d13.up.railway.app')
      })
    })
  }

  return (
    <>
      <div>
        <a href="https://vite.dev" target="_blank">
          <img src={viteLogo} className="logo" alt="Vite logo" />
        </a>
        <a href="https://react.dev" target="_blank">
          <img src={reactLogo} className="logo react" alt="React logo" />
        </a>
      </div>
      <h1>FastAPI + React + Railway</h1>
      
      {/* API Connection Status */}
      <div className="card">
        <h2>🔗 API Connection</h2>
        {isLoading && <p>Loading...</p>}
        {error && <p style={{ color: 'red' }}>❌ {error}</p>}
        {apiMessage && <p style={{ color: 'green' }}>✅ {apiMessage}</p>}
        <div style={{ display: 'flex', gap: '10px', flexWrap: 'wrap' }}>
          <button onClick={fetchApiMessage} disabled={isLoading}>
            Test API Connection
          </button>
          <button onClick={runDiagnostics} disabled={isLoading}>
            Run Diagnostics
          </button>
          <button onClick={testUrlGeneration} disabled={isLoading}>
            Test URL Generation
          </button>
        </div>
      </div>

      {/* App Info */}
      {appInfo && (
        <div className="card">
          <h2>📋 App Information</h2>
          <ul style={{ textAlign: 'left' }}>
            <li><strong>App:</strong> {appInfo.app_name}</li>
            <li><strong>Version:</strong> {appInfo.version}</li>
            <li><strong>Environment:</strong> {appInfo.environment}</li>
            {appInfo.railway_environment && (
              <li><strong>Railway Environment:</strong> {appInfo.railway_environment}</li>
            )}
            <li><strong>Python Version:</strong> {appInfo.python_version}</li>
          </ul>
        </div>
      )}

      {/* Counter Demo */}
      <div className="card">
        <h2>🔢 Counter Demo</h2>
        <button onClick={() => setCount((count) => count + 1)}>
          count is {count}
        </button>
        <p>
          Edit <code>src/App.tsx</code> and save to test HMR
        </p>
      </div>

      {/* ML Prediction Demo */}
      <div className="card">
        <h2>🤖 ML Prediction Demo</h2>
        <button onClick={testPrediction} disabled={isLoading}>
          {isLoading ? 'Making Prediction...' : 'Test ML Prediction'}
        </button>
        {predictionResult && (
          <div style={{ marginTop: '1rem', textAlign: 'left' }}>
            <h3>Prediction Result:</h3>
            <ul>
              <li><strong>Prediction:</strong> {predictionResult.prediction}</li>
              <li><strong>Confidence:</strong> {(predictionResult.confidence * 100).toFixed(1)}%</li>
              <li><strong>Model Version:</strong> {predictionResult.model_version}</li>
            </ul>
          </div>
        )}
      </div>

      <p className="read-the-docs">
        Click on the Vite and React logos to learn more
      </p>
    </>
  )
}

export default App


In [ ]:
%%writefile frontend/src/index.css
:root {
  font-family: system-ui, Avenir, Helvetica, Arial, sans-serif;
  line-height: 1.5;
  font-weight: 400;

  color-scheme: light dark;
  color: rgba(255, 255, 255, 0.87);
  background-color: #242424;

  font-synthesis: none;
  text-rendering: optimizeLegibility;
  -webkit-font-smoothing: antialiased;
  -moz-osx-font-smoothing: grayscale;
}

a {
  font-weight: 500;
  color: #646cff;
  text-decoration: inherit;
}
a:hover {
  color: #535bf2;
}

body {
  margin: 0;
  display: flex;
  place-items: center;
  min-width: 320px;
  min-height: 100vh;
}

h1 {
  font-size: 3.2em;
  line-height: 1.1;
}

button {
  border-radius: 8px;
  border: 1px solid transparent;
  padding: 0.6em 1.2em;
  font-size: 1em;
  font-weight: 500;
  font-family: inherit;
  background-color: #1a1a1a;
  cursor: pointer;
  transition: border-color 0.25s;
}
button:hover {
  border-color: #646cff;
}
button:focus,
button:focus-visible {
  outline: 4px auto -webkit-focus-ring-color;
}

@media (prefers-color-scheme: light) {
  :root {
    color: #213547;
    background-color: #ffffff;
  }
  a:hover {
    color: #747bff;
  }
  button {
    background-color: #f9f9f9;
  }
}


In [ ]:
%%writefile frontend/src/main.tsx
import { StrictMode } from 'react'
import { createRoot } from 'react-dom/client'
import './index.css'
import App from './App.tsx'

createRoot(document.getElementById('root')!).render(
  <StrictMode>
    <App />
  </StrictMode>,
)


In [ ]:
%%writefile frontend/src/vite-env.d.ts
/// <reference types="vite/client" />
